# TP2 - Organización de Datos
#### Notebook principal

<hr>

### Notebooks utilizados:

- ***feature_engineering:*** notebook donde se generarán nuevos features para luego, realizar un proceso de selección de los mejores features para cada modelo.
- ***pre_processing:*** notebook para el manejo de los dataframes.
- ***parameter_tuning:*** notebook donde se tunean los parámetros. Por tratarse de un proceso costoso temporalmente, una vez obtenidos los mejores parámetros, los guardaremos en vez de generarlos cada vez que se quieran generar los resultados.
- ***predict:*** notebook que se encargará de generar el csv con las predicciones finales para el modelo que se le indique.

<hr>


In [1]:
import pandas as pd
import numpy as np

In [7]:
import nbimporter

from pre_processing import load_train
import predict
import feature_engineering
import parameter_tuning

<hr>

# Resultados obtenidos

### Modelo: Regresion lineal

In [3]:
# ...

### Modelo: Regresion logistica

In [3]:
# ...

### Modelo: SVM

In [4]:
# ...

### Modelo: Decision Tree

In [5]:
# ...

### Modelo: RandomForest

In [6]:
# ...

### Modelo: XGBoost

In [3]:
# ...

### Modelo: CatBoost

In [3]:
# ...

### Modelo: LightGBM

In [3]:
# ...

### Modelo: KNN

In [3]:
# ...

### Modelo: Neural Networks

In [3]:
# ...

# testing

In [8]:
df = load_train()

In [11]:
df.head(3)

,titulo,descripcion,direccion,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Avenida Division del Norte 2005,Apartamento,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,2015-08-23,0,0,0,0,0,2273000.0
1,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",AV. MEXICO,Casa en condominio,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,2013-06-28,0,0,0,1,1,3600000.0
2,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Urbi Tonala,Casa,Tonalá,Jalisco,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,2015-10-17,0,0,0,0,0,1200000.0
